## Bipartite grammars

In [16]:
import networkx as nx
import igraph as ig
import leidenalg as la
import numpy as np
import graph_tool.all as gt
import glob
import re
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
import seaborn as sns; sns.set_style('white')
import sys; sys.path.append('../')
from time import time
import logging
from anytree import LevelOrderIter
from statistics import mean
import os
import pickle
import random
from collections import Counter
import pyintergraph as pig

In [2]:
from VRG.src.utils import nx_to_igraph, check_file_exists
from VRG.src.graph_stats import GraphStats
from VRG.src.graph_comparison import GraphPairCompare
from VRG.runner import get_clustering
from VRG.src.Tree import create_tree, dasgupta_cost
from VRG.src.MDL import graph_dl as graph_mdl

sys path:  ['/home/jupyter-ssikdar/Attributed-VRG/notebooks', '/home/jupyter-ssikdar/miniconda3/envs/VRG/lib/python37.zip', '/home/jupyter-ssikdar/miniconda3/envs/VRG/lib/python3.7', '/home/jupyter-ssikdar/miniconda3/envs/VRG/lib/python3.7/lib-dynload', '', '/home/jupyter-ssikdar/miniconda3/envs/VRG/lib/python3.7/site-packages', '/home/jupyter-ssikdar/miniconda3/envs/VRG/lib/python3.7/site-packages/IPython/extensions', '/home/jupyter-ssikdar/.ipython', '../', './../', './../../']


In [3]:
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 50
plt.rcParams['axes.labelsize'] = 20
plt.rcParams['axes.titlesize'] = 20

In [4]:
def load_pickle(fname):
#     logging.error(f'Reading {fname!r}')
    return pickle.load(open(fname, 'rb'))

In [9]:
def dump_pickle(obj, fname):
#     logging.error(f'Reading {fname!r}')
    return pickle.dump(obj, open(fname, 'wb'))

In [13]:
def get_graph(gname: str = 'sample'):
    start_time = time()
    attr_name = ''
    if gname == 'sample':
        g = nx.Graph()
        g.add_nodes_from(range(5), color='blue')
        g.add_nodes_from(range(5, 9), color='red')

        g.add_edges_from([(0, 1), (0, 3), (0, 4),
                          (1, 2), (1, 4), (1, 5),
                          (2, 3), (2, 4), (2, 8),
                          (3, 4),
                          (5, 6), (5, 7), (5, 8),
                          (6, 7), (6, 8),
                          (7, 8)])  # properly labeled
        g.name = 'sample'
        attr_name = 'color'
    elif gname == 'karate':
        g = nx.karate_club_graph()
        attr_name = 'club'
        g.name = 'karate'
    elif gname == 'BA':
        g = nx.barabasi_albert_graph(10, 2, seed=42)
        # g = nx.MultiGraph(g)
        g = nx.Graph()
    elif gname.endswith('.gpickle'):
        g = nx.read_gpickle(gname)
        g.name = Path(gname).stem
    else:
        if gname in ('waterloo', 'grenoble', 'uppsala'):
            g = nx.read_gpickle(f'../snap_data/cleaned/{gname}_lcc_attr.gpickle')
        elif gname in ('polblogs', 'polbooks', 'football', 'bipartite-10-10', 'us-flights',
                       'cora', 'citeseer', 'pubmed'):
            g = nx.read_gml(f'../VRG/input/{gname}.gml')
            attr_name = 'value'
        else:
            path = f'../VRG/input/{gname}.g'
            g = nx.read_edgelist(path, nodetype=int, create_using=nx.Graph())

        g.remove_edges_from(nx.selfloop_edges(g))
        if not nx.is_connected(g):
            nodes_lcc = max(nx.connected_components(g), key=len)
            g = g.subgraph(nodes_lcc).copy()
        name = g.name
        g = nx.convert_node_labels_to_integers(g, label_attribute='orig_label')
        g.name = name

    end_time = round(time() - start_time, 2)
    logging.error(f'Graph: {gname}, n = {g.order():_d}, m = {g.size():_d}, read in {round(end_time, 3):_g}s.')

    return g, attr_name

In [6]:
g, attr_name = get_graph('polbooks')

Graph: polbooks, n = 105, m = 441, read in 0.11s.


In [7]:
def chung_lu(input_graph):
    gen_graph = nx.Graph()
    gen_graph.add_nodes_from(input_graph.nodes(data=True))
    
    stub_list = []
    for n, d in input_graph.degree():
        stub_list.extend([n] * d)
    random.shuffle(stub_list)
    
    for i, u in enumerate(stub_list[: -1]):
        v = stub_list[i + 1]
        gen_graph.add_edge(u, v)

    largest_cc = max(nx.connected_components(gen_graph), key=len)
    return gen_graph

In [15]:
names = ['karate', 'football', 'polbooks', 'us-flights', 
         'cora', 'citeseer','polblogs', 'pubmed']
num_graphs = 10

for name in names:
    input_graph, attr_name = get_graph(name)
    graphs = [chung_lu(input_graph) for _ in range(num_graphs)]
    dump_pickle(graphs, 
                f'/data/ssikdar/attributed-vrg/dumps/graphs/{name}/Chung-Lu_{num_graphs}.pkl')

Graph: karate, n = 34, m = 78, read in 0s.
Graph: football, n = 115, m = 613, read in 0.07s.
Graph: polbooks, n = 105, m = 441, read in 0.04s.
Graph: us-flights, n = 535, m = 2_772, read in 0.26s.
Graph: cora, n = 2_485, m = 5_069, read in 0.71s.
Graph: citeseer, n = 2_110, m = 3_668, read in 0.42s.
Graph: polblogs, n = 1_222, m = 16_714, read in 1.47s.
Graph: pubmed, n = 19_717, m = 44_324, read in 4.99s.


In [12]:
pwd

'/home/jupyter-ssikdar/Attributed-VRG/notebooks'

In [26]:
def sbm(nx_g):
    gt_g = pig.nx2gt(nx_g)
    gt_gen = dc_sbm(gt_g)
    nx_g = nx.Graph(pig.gt2nx(gt_gen))
    return nx_g

In [20]:
def dc_sbm(gt_g):
    assert isinstance(gt_g, gt.Graph)
    g = gt.GraphView(gt_g, vfilt=gt.label_largest_component(gt_g))
    g = gt.Graph(g, prune=True)
    g.set_directed(False)

    state = gt.minimize_blockmodel_dl(g)

    u = gt.generate_sbm(state.b.a, gt.adjacency(state.get_bg(), state.get_ers()).T,
                        g.degree_property_map("total").a,
                        g.degree_property_map("total").a, directed=False)
    return u

In [21]:
g, _ = get_graph('football')

Graph: football, n = 115, m = 613, read in 0.09s.


In [27]:
sbm_g = sbm(g)

In [29]:
print(nx.info(sbm_g))

Name: 
Type: Graph
Number of nodes: 115
Number of edges: 482
Average degree:   8.3826


In [30]:
names = ['karate', 'football', 'polbooks', 'us-flights', 
         'cora', 'citeseer','polblogs', 'pubmed']
num_graphs = 10

for name in names:
    input_graph, attr_name = get_graph(name)
    graphs = [sbm(input_graph) for _ in range(num_graphs)]
    dump_pickle(graphs, 
                f'/data/ssikdar/attributed-vrg/dumps/graphs/{name}/SBM_{num_graphs}.pkl')

Graph: karate, n = 34, m = 78, read in 0s.
Graph: football, n = 115, m = 613, read in 0.05s.
Graph: polbooks, n = 105, m = 441, read in 0.05s.
Graph: us-flights, n = 535, m = 2_772, read in 0.32s.
Graph: cora, n = 2_485, m = 5_069, read in 0.61s.
Graph: citeseer, n = 2_110, m = 3_668, read in 0.39s.
Graph: polblogs, n = 1_222, m = 16_714, read in 1.29s.
Graph: pubmed, n = 19_717, m = 44_324, read in 4.59s.


In [ ]:
orig_g, attr_name = get_graph('pubmed')
deg_ctr = Counter(d for n, d in orig_g.degree())
X, Y = zip(*deg_ctr.items())

cl_g = chung_lu(g)
deg_ctr = Counter(d for n, d in cl_g.degree())
X_cl, Y_cl = zip(*deg_ctr.items())

In [ ]:
ax = plt.gca()
# sns.scatterplot(x=X, y=Y, ax=ax, color='blue', alpha=0.4, label='Original');
sns.scatterplot(x=X_cl, y=Y_cl, ax=ax, color='red', alpha=0.4, label='Chung-Lu');

In [ ]:
nx.degree_assortativity_coefficient(orig_g), nx.degree_assortativity_coefficient(cl_g)

In [ ]:
nx.attribute_assortativity_coefficient(orig_g, attr_name), nx.attribute_assortativity_coefficient(cl_g, attr_name)

## Graph Tool SBM stuff

In [ ]:
g = gt.collection.data["football"]
print(g)

In [ ]:
state = gt.minimize_blockmodel_dl(g)

In [ ]:
state.draw(pos=g.vp.pos);

In [ ]:
e = state.get_matrix().todense()
plt.matshow(e);

In [ ]:
plt.rcParams['figure.figsize'] = (10, 8)
sns.heatmap(e, annot=True, fmt='.3g');

In [ ]:
h_state = gt.minimize_nested_blockmodel_dl(g)

In [ ]:
h_state.draw();

In [ ]:
h_state.print_summary()

In [ ]:
h_state.entropy()

In [ ]:
h_state_ndc = gt.minimize_nested_blockmodel_dl(g, deg_corr=False)
h_state_dc = gt.minimize_nested_blockmodel_dl(g, deg_corr=True)

print(h_state_ndc.entropy(), h_state_dc.entropy())

## Graphtool shuffling

In [ ]:
input_g.num_edges()

In [ ]:
n_iters

In [ ]:
name = 'football'
input_g = gt.collection.data[name]

models = {'Erdos-Renyi': 'erdos', 'CL': 'configuration', 'CL-deg': 'constrained-configuration', 
          'CL-attr': 'constrained-configuration'}
shuffled_graphs = {model: [] for model in models}
n_iters = np.linspace(0, input_g.num_edges(), 10, endpoint=True, dtype=int)

rows = []
orig_deg_ast = gt.scalar_assortativity(input_g, 'total')[0]
orig_attr_ast = gt.scalar_assortativity(input_g, input_g.vp.value)[0]

for model, m in models.items():
    for n_iter in n_iters:
        new_g = input_g.copy()
        
        if model == 'CL-attr':
            gt.random_rewire(g=new_g, model=m, n_iter=n_iter, edge_sweep=False,
                            block_membership=new_g.vp.value)
        else:
            gt.random_rewire(g=new_g, model=m, n_iter=n_iter, edge_sweep=False)
        gen_deg_ast = gt.scalar_assortativity(new_g, 'total')[0]
        gen_attr_ast = gt.scalar_assortativity(new_g, new_g.vp.value)[0]
        
        shuffled_graphs[model].append(new_g)
        rows.append(dict(name=name, model=model, frac=round(n_iter/input_g.num_edges(), 2),
                         orig_graph=input_g, gen_graph=new_g,
                         orig_deg_ast=orig_deg_ast, orig_attr_ast=orig_attr_ast,
                         gen_deg_ast=gen_deg_ast, gen_attr_ast=gen_attr_ast))
        
df = pd.DataFrame(rows)

In [ ]:
df.head()

In [ ]:
plt.rcParams['figure.figsize'] = (12, 6)

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, sharey=True, sharex=True)

ax1.axhline(df.orig_deg_ast.mean(), c='k');
sns.lineplot(x='frac', y='gen_deg_ast', hue='model', marker='o', alpha=0.6, data=df, ax=ax1);
ax1.set_ylabel('Assortativity')
ax1.set_title('Degree');
ax1.legend(loc='best');

ax2.axhline(df.orig_attr_ast.mean(), c='k');
sns.lineplot(x='frac', y='gen_attr_ast', hue='model', marker='o', alpha=0.6, data=df, ax=ax2);
ax2.set_ylabel('Attribute Assortativity')
ax2.set_title('Attribute');

plt.tight_layout();
ax2.legend().set_visible(False);

In [ ]:
print(gt.collection.data.keys())

In [ ]:
g = gt.collection.ns['pgp-strong-2009']
print(g)

In [ ]:
# name = 'malaria_genes/HVR_9'
name = 'openflights'
g = gt.collection.ns[name]
print(g)
print(list(g.vp.keys()))
print(list(g.ep.keys()))

In [ ]:
rows = []

for i in g.vertices():
    rows.append(dict(name=g.vp.name[i], city=g.vp.city[i], country=g.vp.country[i]))

In [ ]:
df = pd.DataFrame(rows)

In [ ]:
df.head()

In [ ]:
df.country.value_counts()

In [ ]:
df[df.country=='United States'].shape

In [ ]:
g.vertex_index

In [ ]:
g.vertex_index[1]

In [ ]:
g_fil = gt.GraphView(g, vfilt=lambda v: g.vp.country[g.vertex_index[v]] == 'United States')

In [ ]:
g_fil.set_directed(False)

In [ ]:
gt.remove_parallel_edges(g_fil)

In [ ]:
g_fil

In [ ]:
state = gt.minimize_blockmodel_dl(g_fil)

In [ ]:
g_fil

In [ ]:
g_fil.save('/data/ssikdar/attributed-vrg/us-airports.graphml')

In [ ]:
! head /data/ssikdar/attributed-vrg/flights/nodes.csv

In [ ]:
nodes_df = pd.read_csv('/data/ssikdar/attributed-vrg/flights/nodes.csv')
edges_df = pd.read_csv('/data/ssikdar/attributed-vrg/flights/edges.csv')

In [ ]:
nodes_df.head()

In [ ]:
nodes_df.columns

In [ ]:
usa_nodes_df = nodes_df[nodes_df.country=='United States']

In [ ]:
nodes_df.head()

In [ ]:
usa_nodes_df.head()

In [ ]:
usa_edges_df = edges_df[(edges_df.source.isin(usa_nodes_df.index)) & (edges_df.target.isin(usa_nodes_df.index))]

In [ ]:
usa_edges_df.head()

In [ ]:
usa_edges_df.shape

In [ ]:
flights_g = nx.Graph()

for row in usa_nodes_df.itertuples():
    flights_g.add_node(row.index, name=row.name, city=row.city)

In [ ]:
flights_g.order()

In [ ]:
for row in usa_edges_df.itertuples():
    flights_g.add_edge(row.source, row.target)

In [ ]:
print(nx.info(flights_g))

In [ ]:
flights_g.remove_edges_from(nx.selfloop_edges(flights_g))

In [ ]:
flights_g.size()

In [ ]:
list(flights_g.nodes(data=True))[: 5]

In [ ]:
from geopy.geocoders import Nominatim

In [ ]:
geolocator = Nominatim(user_agent='blah')
location = geolocator.geocode("Barter Island", addressdetails=True)

In [ ]:
location.raw['address']['state']

In [ ]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}


In [ ]:
def get_state_abv(row):
    location = geolocator.geocode(f'{row.city}, {row.country}', addressdetails=True)
    if location is None or location.raw is None:
        return ''
    state = location.raw['address'].get('state', '')
    return us_state_abbrev.get(state, '')

In [ ]:
usa_nodes_df['state'] = usa_nodes_df.apply(lambda row: get_state_abv(row), axis=1)

In [ ]:
usa_nodes_df = usa_nodes_df[usa_nodes_df.state!='']

In [ ]:
usa_nodes_df

In [ ]:
usa_nodes_df_copy = usa_nodes_df.copy()

In [ ]:
usa_nodes_df.to_csv('/data/ssikdar/attributed-vrg/flights/us_nodes.csv', index=False)

In [ ]:
usa_edges_df = edges_df[(edges_df.source.isin(usa_nodes_df.index)) & (edges_df.target.isin(usa_nodes_df.index))]

In [ ]:
usa_edges_df.shape

In [ ]:
regions_dict = {
    'New England': ['CT', 'ME', 'MA', 'NH', 'RI', 'VT'], 
    'Mideast': ['DE', 'DC', 'MD', 'NJ', 'NY', 'PA'],  
    'Great Lakes': ['IL', 'IN', 'MI', 'OH', 'WI'],  
    'Plains': ['IA', 'KS', 'MN', 'MO', 'NE', 'ND', 'SD'], 
    'Southeast': ['AL', 'AR', 'FL', 'GA', 'KY', 'LA', 'MS', 'NC', 'SC', 'TN', 'VA', 'WV'], 
    'Southwest': ['AZ', 'NM', 'OK', 'TX'],  
    'Rocky Mountain': ['CO', 'ID', 'MT', 'UT', 'WY'], 
    'Far West': ['AK', 'CA', 'HI', 'NV', 'OR', 'WA']  
}

In [ ]:
state2regions = {state: reg for reg, states in regions_dict.items() for state in states}

In [ ]:
len(state2regions)

In [ ]:
usa_nodes_df['region'] = usa_nodes_df.state.apply(lambda x: state2regions[x])

In [ ]:
usa_nodes_df

## SBM

In [ ]:
def dc_sbm(gt_g):
    assert isinstance(gt_g, gt.Graph)
    g = gt.GraphView(gt_g, vfilt=gt.label_largest_component(gt_g))
    g = gt.Graph(g, prune=True)
    g.set_directed(False)
    
    state = gt.minimize_blockmodel_dl(g)
    
    u = gt.generate_sbm(state.b.a, gt.adjacency(state.get_bg(), state.get_ers()).T,
                        g.degree_property_map("total").a,
                        g.degree_property_map("total").a, directed=False)
    return u

In [ ]:
g = gt.collection.data["football"]
sbm_g = dc_sbm(g)

# gt.graph_draw(g, g.vp.pos)
# gt.graph_draw(u, u.own_property(g.vp.pos))

In [ ]:
gt.graph_draw(g, g.vp.pos);

In [ ]:
gt.graph_draw(sbm_g, sbm_g.own_property(g.vp.pos));

In [ ]:
g, sbm

In [ ]:
u